In [ ]:
!pip install transformers peft datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 54.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "openai-community/gpt2"

model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def run(text):
    inputs = tokenizer(text, return_tensors="pt")
    output = model.generate(**inputs)
    return tokenizer.decode(output[0])

run('Hello')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


"Hello, I'm sorry, but I'm not sure if you're aware of this. I'm"

In [ ]:
from datasets import load_dataset

english_data = load_dataset('Abirate/english_quotes', split="train[:1000]")
spanish_data = load_dataset('andreamorgar/spanish_poetry', split="train[:1000]")
french_data = load_dataset('Abirate/french_book_reviews', split="train[:1000]")

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [ ]:
spanish_data['content'][0]

'\n\nEn el parque confuso\nQue con lánguidas brisas el cielo sahúma,\nEl ciprés, como un huso,\nDevana un ovillo de de bruma.\nEl telar de la luna tiende en plata su urdimbre;\nAbandona la rada un lúgubre corsario,\nY después suena un timbre\nEn el vecindario.\n\nSobre el horizonte malva\nDe una mar argentina,\nEn curva de frente calva\nLa luna se inclina,\nO bien un vago nácar disemina\nComo la valva\nDe una madreperla a flor del agua marina.\n\nUn brillo de lóbrego frasco\nAdquiere cada ola,\nY la noche cual enorme peñasco\nVa quedándose inmensamente sola.\n\nForma el tic-tac de un reloj accesorio,\nLa tela de la vida, cual siniestro pespunte.\nFlota en la noche de blancor mortuorio\nUna benzoica insispidez de sanatorio,\nY cada transeúnte\nParece una silueta del Purgatorio.\n\nCon emoción prosaica,\nSuena lejos, en canto de lúgubre alarde,\nUna voz de hombre desgraciado, en que arde\nEl calor negro del rom de Jamaica.\nY reina en el espíritu con subconsciencie arcaica,\nEl miedo de 

In [ ]:
def preprocess_data(examples, column):
    return tokenizer(
        [x for x in examples[column] if x],
        truncation=True,
        max_length = 512,
        padding=True
    )

tokenized_english = english_data.map(
    lambda examples: preprocess_data(examples,'quote'),
    batched=True,
    num_proc=4,
    remove_columns=english_data.column_names,
)

tokenized_spanish = spanish_data.map(
    lambda examples: preprocess_data(examples,'content'),
    batched=True,
    num_proc=4,
    remove_columns=spanish_data.column_names,
)

tokenized_french = french_data.map(
    lambda examples: preprocess_data(examples,'reader_review'),
    batched=True,
    num_proc=4,
    remove_columns=french_data.column_names,)

In [ ]:
# def preprocess_english(examples):
#     return tokenizer(
#         [x for x in examples['quote'] if x],
#         truncation=True,
#         max_length = 512,
#         padding=True
#     )


# def preprocess_spanish(examples):
#     return tokenizer(
#         [x for x in examples['content'] if x],
#         truncation=True,
#         max_length = 512,
#         padding=True
#     )

# def preprocess_french(examples):
#     return tokenizer(
#         [x for x in examples['reader_review'] if x],
#         truncation=True,
#         max_length = 512,
#         padding=True
#     )

# tokenized_english = english_data.map(
#     preprocess_english,
#     batched=True,
#     num_proc=4,
#     remove_columns=english_data.column_names,
# )

# tokenized_spanish = spanish_data.map(
#     preprocess_spanish,
#     batched=True,
#     num_proc=4,
#     remove_columns=spanish_data.column_names,
# )

# tokenized_french = french_data.map(
#     preprocess_french,
#     batched=True,
#     num_proc=4,
#     remove_columns=french_data.column_names,)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:

tokenizer

GPT2TokenizerFast(name_or_path='openai-community/gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(r=32,lora_alpha=20,task_type="CAUSAL_LM", # LoRA attention dimension)


In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

English

In [ ]:
model.add_adapter(lora_config, adapter_name='english_adapter_lm')

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_english,
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=375, training_loss=4.032800455729166, metrics={'train_runtime': 329.7168, 'train_samples_per_second': 9.099, 'train_steps_per_second': 1.137, 'total_flos': 800183549952000.0, 'train_loss': 4.032800455729166, 'epoch': 3.0})

In [ ]:
model.save_pretrained(save_directory='english_adapter_dir')

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:399: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Spanish

In [ ]:
model.add_adapter(lora_config, adapter_name='spanish_adapter_lm')

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1119: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_spanish,
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=375, training_loss=4.7470283203125, metrics={'train_runtime': 341.9992, 'train_samples_per_second': 8.772, 'train_steps_per_second': 1.096, 'total_flos': 789311913984000.0, 'train_loss': 4.7470283203125, 'epoch': 3.0})

In [ ]:
model.save_pretrained(save_directory='spanish_adapter_dir')

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:399: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


French

In [ ]:
model.add_adapter(lora_config, adapter_name='french_adapter_lm')

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_french,
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=375, training_loss=4.878352864583333, metrics={'train_runtime': 334.7813, 'train_samples_per_second': 8.961, 'train_steps_per_second': 1.12, 'total_flos': 794747731968000.0, 'train_loss': 4.878352864583333, 'epoch': 3.0})

In [ ]:
model.save_pretrained(save_directory='french_adapter_dir')

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:399: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
model_id = "openai-community/gpt2"
model_2 = AutoModelForCausalLM.from_pretrained(model_id)

model_2.load_adapter(
    peft_model_id='spanish_adapter_dir',
    adapter_name='spanish_adapter'
)

model_2.load_adapter(
    peft_model_id='french_adapter_dir',
    adapter_name='french_adapter'
)

model_2.load_adapter(
    peft_model_id='english_adapter_dir',
    adapter_name='english_adapter'
)


In [ ]:
customer_to_adapter_lookup = {
    'spanish': 'spanish_adapter_lm',
    'french': 'french_adapter_lm',
    'english': 'english_adapter_lm'
}

In [ ]:
def run(data):

    # get the customer ID
    customer_id = data['customer_id']

    text = data['input']

    # get the adapter of that customer
    adapter = customer_to_adapter_lookup[customer_id]
    print(adapter)

    # switch to customer's adapter
    model.set_adapter(adapter)

    inputs = tokenizer(text, return_tensors="pt")

    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    output = model.generate(**inputs)
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
data = {
    'customer_id': 'spanish',
    'input': 'parque confuso'
}

run(data)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


spanish_adapter_lm


'parque confuso, et, et et, et, et, et, et, et'